In [ ]:
from tqdm import tqdm
from D3QNAgent import Agent
from kaggle_environments import evaluate, make, utils
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

env = make("connectx", debug=True)
env.render()

nstep_lookahead_agent = './fast_Nstep_lookahead_agent.py'
random_agent = 'random'
negamax_agent = 'negamax'

In [ ]:
def live_plot(rewards, winloss):
    clear_output(wait=True)
    fig, (ax1, ax2) = plt.subplots(ncols=2, dpi=120, figsize=(12.8, 3.6))
    ax1.plot(np.array(rewards), color='orange', linestyle='-')
    # ax1.legend(loc='lower right')
    ax1.set_ylabel('reward')
    ax1.set_xlabel('episode')
    ax1.title.set_text('Agent Total Rewards with Moving Average 100 Episodes')
    

    # ax2.plot(np.array(winloss), color='royalblue', linestyle='-')
    # # ax2.legend(loc='lower right')
    # ax2.set_ylabel('Expectation')
    # ax2.set_xlabel('episode')
    # ax2.title.set_text('Win/Loss/Draw Expectation with Moving Average 100 Episodes')
    plt.show()

    print('episode', i, 
        # '\ttotal rewards %.1f' % tot_reward, 
        '\taverage rewards %.3f' % avg_rewards[i], 
        # '\tresult ', reward, 
        # '\twin/loss ratio %.3f' % wl_ratio[i], 
        '\tepsilon %.3f' % agent.epsilon)


In [ ]:
agent = Agent(
    env.configuration, 
    lr=1e-4, 
    gamma=0.95, 
    batch_size=64,    #### REMEMBER TO CHANGE IT BACK ####
    epsilon=1,        #### REMEMBER TO CHANGE IT BACK ####
    eps_dec=0.9995, 
    eps_min=0.1, 
    buff_size=100_000, 
    d1_dims=128, 
    d2_dims=128,
    replace_target_weight=10)

In [ ]:

rewards, avg_rewards, result, eps_history, wl_ratio = [], [], [], [], []

# None means the agent being trained
trainer = env.train([negamax_agent, None])

In [ ]:

num_episodes = 30000


In [ ]:
for i in range(1):
    done = False
    tot_reward = 0

    observation = trainer.reset()
    board = observation['board']
    marker = observation.mark
    obs = np.array(board, dtype=np.float32).reshape(6, 7, 1)

    if marker == 1:
        obs[obs == 2] = -10
        obs[obs == 1] = 10
    else:
        obs[obs == 1] = -10
        obs[obs == 2] = 10
    
    # print(obs.reshape(6, 7))

    while not done:

        # choose the best action
        action = agent.choose_action(obs)
        print("action: ", action)
        
        # step the environment with action
        # store all returns
        observation_, reward, done, info = trainer.step(action)
        board_ = observation_['board']
        marker_ = observation_.mark
        obs_ = np.array(board_, dtype=np.float32).reshape(6, 7, 1)
        if marker_ == 1:
            obs_[obs_ == 2] = -10
            obs_[obs_ == 1] = 10
        else:
            obs_[obs_ == 1] = -10
            obs_[obs_ == 2] = 10

        # print(obs_.reshape(6, 7))

        # calculate agent reward from environment response
        agent_reward = agent.get_agent_reward(reward, done)
        
        tot_reward += agent_reward
        
        # store this transition
        agent.update_replay_buffer((obs, action, reward, obs_, done))
        
        # update the current obs with new obs
        obs = obs_
        
        
        agent.learn()
    
    agent.evolve()

    # store final game result
    result.append(reward)
    # store epsilon
    eps_history.append(agent.epsilon)
    # store total rewards
    rewards.append(tot_reward)
    # store average rewards
    avg_rewards.append(np.mean(rewards[-100:]))
    wl_ratio.append(np.mean(result[-100:]))
    


    # if i % 50 == 0:
    #     live_plot(avg_rewards, wl_ratio)
        
    env.render()
    
# live_plot(avg_rewards, wl_ratio)

In [ ]:
agent.save_model('./models/negamax_30000_conv')

In [ ]:
from D3QNAgent import Agent
def my_agent(obs, config):
    # define agent
    agent = Agent(config)
    agent.load_model('./models/negamax_30000_conv')

    # preprocessing of the state
    board = obs['board']
    marker = obs.mark
    state = np.array(board, dtype=np.float32).reshape(6, 7, 1)

    if marker == 1:
        obs[obs == 2] = -10
        obs[obs == 1] = 10
    else:
        obs[obs == 1] = -10
        obs[obs == 2] = 10

    return agent.choose_action(state)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run([my_agent, nstep_lookahead_agent])
env.render(mode="ipython", width=500, height=450)

In [ ]:
# def mean_reward(rewards):
#     return sum(r[0] for r in rewards) / float(len(rewards))
    
# print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=10)))